In [2]:
import torch


# Example usage
N, T, B = 2, 5, 16
in_features, out_features = B//4, 10

# create input tensor of shape [NxTxB/4]
x = torch.randn(N, T, in_features)

x = x.view(-1, x.shape[-1])


In [3]:
x.shape

torch.Size([10, 4])

In [4]:

# apply linear layer to each time step
x = torch.nn.Linear(in_features, out_features)(x)


In [5]:
x.shape

torch.Size([10, 10])

In [6]:

# reshape output tensor to [N x T x S]
x = x.view(*x.shape[:-1], -1)

In [7]:
x.shape

torch.Size([10, 10])

In [8]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class TimeDistributed(nn.Module):
    def __init__(self, in_features, out_features):
        super(TimeDistributed, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        # reshape input tensor to [N*T x B/4]
        x = x.view(-1, x.shape[-1])

        # apply linear layer to each time step
        x = self.linear(x)

        # reshape output tensor to [N x T x S]
        x = x.view(*x.shape[:-1], -1)

        return x


# Example usage
N, T, B = 2, 5, 16
in_features, out_features = B//4, 10

# create input tensor of shape [NxTxB/4]
x = torch.randn(N, T, in_features)

# pack input tensor and apply time-distributed linear layer
packed_x = rnn_utils.pack_padded_sequence(x, lengths=[T]*N, batch_first=True, enforce_sorted=False)
linear_layer = TimeDistributed(in_features, out_features)
y, _ = rnn_utils.pad_packed_sequence(linear_layer(packed_x)[0], batch_first=True)

# output tensor of shape [N x T x S]
print(y.shape)  # should be [2, 5, 10]


AttributeError: 'PackedSequence' object has no attribute 'view'

In [14]:
class TimeDistributedFC(nn.Module):
    """
    Class for applying time distributed fully connected layers with input tensors 
    of shape  [N, T, in_features] where N is the batch_size, T is the number of time step,
    and in_features is the number of input features at each time step. 
    Output tensors will have shape [N, T, out_features], where out_features is the number 
    of output features at each time step.
    """
    def __init__(self, in_features, out_features):
        super(TimeDistributedFC, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        
        N,T = x.shape[0],x.shape[1]
        
        # reshape input tensor to [N*T x B/4]
        x = x.view(-1, x.shape[-1])

        # apply linear layer to each time step
        x = self.linear(x)

        # reshape output tensor to [N x T x S]
        x = x.view(N, T,-1)

        return x

In [15]:
# Example usage
N, T, B = 2, 5, 16
in_features, out_features = B//4, 10

# create input tensor of shape [NxTxB/4]
x = torch.randn(N, T, in_features)

# # pack input tensor and apply time-distributed linear layer
# packed_x = rnn_utils.pack_padded_sequence(x, lengths=[T]*N, batch_first=True, enforce_sorted=False)
linear_layer = TimeDistributedFC(in_features, out_features)
# y, _ = rnn_utils.pad_packed_sequence(linear_layer(packed_x)[0], batch_first=True)
y = linear_layer(x)
print(y.shape)

torch.Size([2, 5, 10])
